# Table of Contents
 <p>

Reference: http://satomacoto.blogspot.jp/2009/12/pythonlda.html   
Code in the reference is modified for Python 3  

LDA ([Blei et al.](http://www.jmlr.org/papers/volume3/blei03a/blei03a.pdf) Figure 1)  
Variational Bayes EM

In [1]:
import sys,getopt
from numpy import array,matrix,diag
from scipy import sum,log,exp,mean,dot,ones,zeros
from scipy.special import polygamma
from scipy.linalg import norm
from random import random
import os
import pandas as pd

In [2]:
os.chdir("/home/3928941380/Downloads/")

In [3]:
def main():
    # set parameters
    k = 10 # of classes to assume
    emmax = 100 # of maximum VB-EM iteration (default 100)
    demmax = 20 # of maximum VB-EM iteration for a document
    epsilon = 0.0001 # A threshold to determine the whole convergence of the estimation
    
    # Train
    train = open("train.txt",'r').read()
    alpha,phi = ldamain(train, k, emmax, demmax, epsilon)
    
    # Write
    writer = open('output-alpha.txt','w')
    writer.write(str(alpha.tolist()))
    writer.close() 
    
    writer = open('output-phi.txt','w')
    writer.write(str(phi.tolist()))
    writer.close()
    
    return alpha, phi

In [4]:
def ldamain(train, k, emmax=100, demmax=20, epsilon=1.0e-4):
    d = [ zip(*[ [int(x) for x in w.split(':')] for w in L.split()]) for L in train.split('\n') if L ]
    
    data = []
    for L in train.split("\n"):
        if L == "":
            continue

        id_ = [int(w.split(":")[0]) for w in L.split(" ")]
        w_count = [int(w.split(":")[1]) for w in L.split(" ")]

        data.append([id_, w_count])
    
    return lda.train(data,k,emmax,demmax)

In [7]:
train = open("train.txt",'r').read()
d = [ zip(*[ [int(x) for x in w.split(':')] for w in L.split()]) for L in train.split('\n') if L ]

data = []
for L in train.split("\n"):
    if L == "":
        continue

    id_ = [int(w.split(":")[0]) for w in L.split(" ")]
    w_count = [int(w.split(":")[1]) for w in L.split(" ")]

    data.append([id_, w_count])
    
d = data

In [13]:
k = 10 # of classes to assume
emmax = 100 # of maximum VB-EM iteration (default 100)
demmax = 20 # of maximum VB-EM iteration for a document
epsilon = 0.0001 # A threshold to determine the whole convergence of the estimation

# # of documents
M = len(d)
# # of words
L = max(map(lambda x: max(x[0]), d)) + 1

# initialize
phi = matrix(ones((k, L)) / L)
alpha = matrix(lda.normalize(sorted([random() for i in range(k)], reverse=True))).T
gammas = matrix(zeros((M, k)))
lik = 0
plik = lik
phis = matrix(zeros((k, L)))

In [14]:
di, phi, alpha0 = d[1], phi, alpha

In [15]:
matrix(phi[:,di[0]]).shape

(10, 88)

In [16]:
alpha0.shape

(10, 1)

In [17]:
nt.shape

(10, 1)

In [18]:
digamma(alpha0 + nt).shape

(10, 1)

In [19]:
exp(digamma(alpha0 + nt)).shape

(10, 1)

In [20]:
diag(exp(digamma(alpha0 + nt))[0])

array([[ 132.19827725]])

In [21]:
matrix(phi[:,di[0]]).shape

(10, 88)

In [22]:
di, phi, alpha0 = d[0], phi, alpha
digamma = lambda x: polygamma(0,x)

digamma = lambda x: polygamma(0,x)

Nd = len(di[0])
k = len(alpha0)
q = zeros((Nd, k))
nt = matrix(ones((1, k)) * L / k).T
pnt = nt


    #vb-estep
q = matrix( matrix(phi[:,di[0]]) * diag(exp(digamma(alpha0 + nt))[:,0]))
q = lda.mnormalize(q , 0)

ValueError: shapes (10,69) and (10,10) not aligned: 69 (dim 1) != 10 (dim 0)

In [74]:
gammas[1,:].shape

(10,)

In [77]:
matrix(zeros((M, k)))[1,:].shape

(1, 10)

In [87]:
matrix(lda.normalize(sorted([random() for i in range(k)], reverse=True))).T.shape

(10, 1)

In [132]:
alpha.shape

(10, 1)

In [125]:
nt.shape

(10, 1)

In [144]:
nt.shape

(10, 1)

In [142]:
len(d)

100

In [143]:
def fpi_alpha(alpha, nt_keep):
    # fixed point iteration for alpha / Sato p.112
    K = alpha.shape[0]
    M = len(nt_keep)

    for k in range(K):
        alpha_k = alpha[k, 0]

        numerator = 0 ; denominator = 0
        for m in range(M):
            nd = nt[m].sum() # Sato p.114

            # numerator
            numerator += (digamma(nt[k,0] + alpha_k) - digamma(alpha_k)) * alpha_k

            # denominator
            denominator += digamma(nd + alpha.sum()) - digamma(alpha.sum())

        alpha[k,0] = numerator / denominator

    return alpha

In [11]:
# # of documents
M = len(d)
# # of words
L = max(map(lambda x: max(x[0]), d)) + 1

# initialize
phi = matrix(ones((k, L)) / L)
alpha = matrix(lda.normalize(sorted([random() for i in range(k)], reverse=True))).T
gammas = matrix(zeros((M, k)))
lik = 0
plik = lik

phis = matrix(zeros((k, L)))
nt_keep = []
for i in range(M):
    gamma, q, nt = lda.vbem(d[i], phi, alpha, demmax)
    nt_keep.append(nt)
    gammas[i,:] = gamma.T
    phis = lda.accum_phi(phis,q,d[i])

In [41]:
def mnormalize(m, d=0):
    '''
    x = mnormalize(m, d)
    normalizes a 2-D matrix m along the dimension d.
    m : matrix
    d : dimension to normalize (default 0)
    '''
    m = array(m)
    v = sum(m, d)
    if d == 0:
        return m * matrix(diag(1.0 / v))
    else:
        return matrix(diag(1.0 / v)) * m

In [42]:
egamma = matrix(mnormalize(gammas, 1))

In [43]:
egamma.shape

(100, 10)

In [157]:
len(nt_keep)

100

In [161]:
len(nt_keep[1])

9

In [ ]:
@staticmethod
def lda_lik(d, phi, gammas):
    '''
    lik = lda_lik(d, phi, gammas)
    returns the likelihood of d, given LDA model of (phi, gammas).
    '''
    egamma = matrix(lda.mnormalize(gammas, 1))
    lik = 0
    M = len(d)
    for i in range(M):
        t = d[i]
        lik += (matrix(t[1]).T * log(matrix(phi[:,t[0]]) * egamma[i,:].T))[0,0]
    return lik

In [45]:
t = d[1]

In [53]:
(matrix(t[1]) * log(matrix(phi[:,t[0]]).T * egamma[i,:].T))[0,0]

nan

In [60]:
matrix(t[1]).shape

(1, 88)

In [59]:
log(matrix(phi[:,t[0]]).T * egamma[i,:].T).shape

(88, 1)

In [50]:
matrix(phi[:,t[0]]).shape

(10, 88)

In [52]:
egamma[i,:].T.shape

(10, 1)

In [34]:
gamma.shape

(10, 1)

In [58]:
gammas.shape

(100, 10)

In [36]:
egamma.shape

(100, 10)

In [61]:
class lda():
    '''
    Latent Dirichlet Allocation, standard model.
    [alpha,phi] = lda.train(d,k,[emmax,demmax])
    d      : data of documents
    k      : # of classes to assume
    emmax  : # of maximum VB-EM iteration (default 100)
    demmax : # of maximum VB-EM iteration for a document (default 20)
    '''
    
    @staticmethod
    def train(d, k, emmax=100, demmax=20, epsilon=1.0e-4):
        '''
        Latent Dirichlet Allocation, standard model.
        [alpha,phi] = lda.train(d,k,[emmax,demmax])
        d      : data of documents
        k      : # of classes to assume
        emmax  : # of maximum VB-EM iteration (default 100)
        demmax : # of maximum VB-EM iteration for a document (default 20)
        '''
        
        # # of documents
        M = len(d)
        # # of words
        L = max(map(lambda x: max(x[0]), d)) + 1
        
        # initialize
        phi = matrix(ones((k, L)) / L)
        alpha = matrix(lda.normalize(sorted([random() for i in range(k)], reverse=True))).T
        gammas = matrix(zeros((M, k)))
        lik = 0
        plik = lik
        
        #print ('number of documents (M)      = {0}'.format(M))
        #print ('number of words (l)          = {0}'.format(L))
        #print ('number of latent classes (k) = {0}'.format(k))
        
        for j in range(emmax):
            print ('iteration {0}/{1}..\t'.format(j+1, emmax))
            #vb-esstep
            phis = matrix(zeros((k, L)))
            nt_keep = []
            for i in range(M):
                gamma, q, nt = lda.vbem(d[i], phi, alpha, demmax)
                nt_keep.append(nt)
                gammas[i,:] = gamma.T
                phis = lda.accum_phi(phis,q,d[i])
            #vb-mstep
            alpha = lda.fpi_alpha(alpha, nt_keep)
                    #alpha = lda.newton_alpha(gammas)
            phi = lda.mnormalize(phis,1)
            #converge?
            lik = lda.lda_lik(d, phi, gammas)
            print ('log-likelihoood =', lik)
            if j > 1 and abs((lik - plik) / lik) < epsilon:
                if j < 5:
                    print
                    return lda.train(d, k, emmax, demmax) # try again
                    return
                print ('converged')
                return alpha, phi
            plik = lik
                

    @staticmethod
    def vbem(di, phi, alpha0, emmax=20):
        '''
        [alpha,q] = vbem(d,phi,alpha0,[emmax])
        calculates a document and words posterior for a document d.
        alpha  : Dirichlet posterior for a document d
        q      : (L * K) matrix of word posterior over latent classes
        di      : document data / here, only one sentence
        phi   : 
        alpha0 : Dirichlet prior of alpha
        emmax  : maximum # of VB-EM iteration.
        '''
        digamma = lambda x: polygamma(0,x)

        Nd = len(di[0])
        k = len(alpha0)
        q = zeros((Nd, k))
        nt = matrix(ones((1, k)) * Nd / k).T
        pnt = nt

        for j in range(emmax):
            #vb-estep
            q = matrix( matrix(phi[:,di[0]]).T * diag(exp(digamma(alpha0 + nt))[:,0]))
            q = lda.mnormalize(q , 1)
                    # q(z_d,i =k)
                    # alpha0 + ntでよくわからないけど、alphaをアップデートしてるからe-step?
                    # better to look at original C code
                       #  ap[k]というのが、式のexpの分子部分相当
                        #  それにphiをかけて最後に正規化している
            #vb-mstep
            nt =  q.T * matrix(di[1]).T
                    # nt:  probably expectation part in Sato Equation(3.89), same as bottom in Sato p.75
                    #  Sato p.75にあるように本来ならば文章中の一語ずつチェックするが、データセットには
                    #  各単語ごとの出現回数しかないので、単語が登場するたびに足すのではなく、単語の出現回数×qをして
                    #  足しあわせている
                    # better to look at original C code
                
            #converge?
            if j > 1 and lda.converged(nt, pnt, 1.0e-2):
                break
            pnt = nt.copy()
            
        alpha = alpha0 + nt # corresponds to Sato Eq (3.89) / dの全てのkに対して一度にしている
                                        # nt:  probably expectation part in Sato Equation(3.89)
        return alpha, q, nt
    
    @staticmethod
    def fpi_alpha(alpha, nt_keep):
        # fixed point iteration for alpha / Sato p.112
        K = alpha.shape[0]
        M = len(nt_keep)
        digamma = lambda x: polygamma(0,x)

        for k in range(K):
            alpha_k = alpha[k, 0]

            numerator = 0 ; denominator = 0
            for m in range(M):
                nt = nt_keep[m]
                nd = nt.sum() # Sato p.114

                # numerator
                numerator += (digamma(nt[k,0] + alpha_k) - digamma(alpha_k)) * alpha_k

                # denominator
                denominator += digamma(nd + alpha.sum()) - digamma(alpha.sum())

            alpha[k,0] = numerator / denominator

        return alpha

    @staticmethod
    def accum_phi(phis, q, t):
        '''
        phis = accum_phi(phis,q,t)
        accumulates word posteriors to latent classes.
        phis : (V * K) matrix of summand
        q     : (L * K) matrix of word posteriors
        t     : document of struct array
        '''
        phis[: , t[0]] +=  q.T  * matrix(diag(t[1])).T
           # phis[t[0],:].shape 
                # this matrix is (number of unique words in document no1) ¥times  (number of class (category) )
                # t[0] shows word id_
        return phis
    
    @staticmethod
    def lda_lik(d, phi, gammas):
        '''
        lik = lda_lik(d, phi, gammas)
        returns the likelihood of d, given LDA model of (phi, gammas).
        '''
        egamma = matrix(lda.mnormalize(gammas, 1))
        lik = 0
        M = len(d)
        for i in range(M):
            t = d[i]
            lik += (matrix(t[1]) * log(matrix(phi[:,t[0]]).T * egamma[i,:].T))[0,0]
        return lik
    
    @staticmethod
    def normalize(v):
        return v / sum(v)
    
    @staticmethod
    def mnormalize(m, d=0):
        '''
        x = mnormalize(m, d)
        normalizes a 2-D matrix m along the dimension d.
        m : matrix
        d : dimension to normalize (default 0)
        '''
        m = array(m)
        v = sum(m, d)
        if d == 0:
            return m * matrix(diag(1.0 / v))
        else:
            return matrix(diag(1.0 / v)) * m
        
    @staticmethod
    def converged(u, udash, threshold=1.0e-3):
        '''
        converged(u,udash,threshold)
        Returns 1 if u and udash are not different by the ratio threshold
        '''
        return norm(u - udash) / norm(u) < threshold

In [ ]:
if __name__ == '__main__':
    alpha, phi = main()
    
    res_b = pd.DataFrame(phi)
    for i in range(phi.shape[0]):
        res_temp = res_b.ix[i, :].sort_values(ascending=False)[:10]
        res_temp.index += 1
        print("Topic: ", i)
        print(res_temp)
        print("\n")